# Bildklassifikation

Bildklassifikation ist eines der Grundprobleme des Bereichs **Computer Vision**. Es bildet die Basis für viele fortgeschrittenere Aufgaben, wie z.B. Objekterkennung. 

Die Ausgabe eines Bildklassifizierers ist eine einzelne Klassenbezeichnung und ein Konfidenzwert. Die Bildklassifizierung ist nützlich, wenn Sie nur wissen müssen, zu welcher Klasse ein Bild gehört, und nicht wissen müssen, wo sich die Objekte dieser Klasse befinden oder welche Form sie genau haben.

![Image](../beispielbilder/grafiken/image-classification-deeplearning.jpg)

In diesem Notebook werden Sie selbstständig einen Bildklassifizierer auf einem eigenen Datensatz trainieren und evaluieren.

Um das Verständnis zu vertiefen und um auf wichtige Aspekte aufmerksam zu machen haben wir eine Reihe von Fragen und Aufgaben in dieses Notebook eingebaut:

**Beispiel**

1. **Verständnisfragen**

<div class="alert alert-block alert-info">
<b>Frage:</b> In den blauen Boxen stehen Verständnisfragen</div>

2. **Arbeitsaufträge**

<div class="alert alert-block alert-success">
<b>Auftrag:</b> In grünen Boxen finden sich Arbeitsaufträge.</div>

3. **Optionale Inhalte**

<div class="alert alert-block alert-warning">
<b>Optional:</b> In gelben Boxen finden Sie optionale Inhalte.</div>

Besonders komplizierte Aspekte oder Bereiche, in denen leicht Fehler gemacht werden können sind mit einer roten Box gekennzeichnet:

<div class="alert alert-block alert-danger">
<b>Achtung:</b> Hier müssen Sie gut aufpassen. Nehmen Sie sich ausreichend Zeit, die nachfolgenden Angaben zu lesen oder Fragen Sie ein Mitglied des Teams um Hilfe.</div>

**Was *ist* Bildklassifikation?**

Da es sich hierbei um ein Problem des **überwachten Lernens** <a href="https://de.wikipedia.org/wiki/%C3%9Cberwachtes_Lernen"> (zur Auffrischung) </a> handelt, reicht es nicht einfach, dem Algorithmus nur die Bilddaten zuzuführen. Damit eine Zuordnung von Bild &rarr; Label stattfinden kann, müssen die Daten entsprechend **annotiert** sein. 

*Wichtig*: Nicht **alle** Spielarten der Bildklassifizierung sind Teil des überwachten Lernens. Die gängigsten Formen sind es aber, und so auch die Aufgabe, die Sie heute lösen werden.

<details>
    <summary><b>Exkurs: Algorithmendesign vs. Datenkuration</b></summary>
    <h5>Wo liegen heute die Constraints?</h5>

In vielen Bereichen des Deep Learning spielt das Design der Algorithmen heute nicht mehr die Hauptrolle. Insbesondere in der maschinellen              Bildverarbeitung, also <b>Computer Vision</b> sind die <i>besten</i> (oder zumindest: sehr gute) Algorithmen heute bekannt. Der Constraint für die     Performance der Modelle liegt also heute weniger im Bereich der Architektur, sondern mehr in der Qualität der Datensätze. 
    
Ex-Tesla AI Direktor und Gründungsmitglied von OpenAI, Andrej Karpathy, zum Thema Datensatzqualität: <a href="https://youtu.be/PlLmdTcsAWU?si=7QmHbQ5oHrBP2hI6&t=2341"> Zum Interview (ab ca. Min 39:00)</a>
    
    
</details>
<br>

In diesem Notebook widmen wir uns also zwei Dingen:

1. **Dem Sammeln von Bilddaten**
2. **Finetuning eines SOTA-Bildklassifikators**

*SOTA = State-of-the-art*

## 1.1. Importieren der notwendigen Bibliotheken und Funktionen

Um das Bildklassifikations-Problem zu lösen, greifen wir auf ein **vortrainiertes** Modell der YOLO-Reihe zurück.
Eigentlich für die Modelle zur Objekterkennung bekannt, bietet YOLO auch Modelle für die Klassifizierung von Bildern. YOLO steht für:

* **Y**ou
* **O**nly
* **L**ook
* **O**nce

Die YOLO-Modellarchitektur ermöglicht die schnelle und effiziente Erkennung von Objekten in Bildern und Videos. Im Gegensatz zu anderen Methoden benötigt YOLO nur einen Durchgang, um alle Objekte zu identifizieren. Es berücksichtigt unterschiedliche Objektgrößen und bietet Vielseitigkeit für verschiedene Anwendungen. YOLO zeichnet sich durch seine Geschwindigkeit und Fähigkeit aus, komplexe Szenen in einem Schritt zu erfassen.

Wenn Sie mehr über die YOLO-Modelle lesen möchten, finden Sie <a href="https://blog.roboflow.com/guide-to-yolo-models/">hier</a> einen guten Blog-Artikel und <a href="https://docs.ultralytics.com/">hier</a> die Ultralytics-Dokumentation zum YOLOv8-Modell.

Zur Verwendung des Modells und für das Aufbereiten der Bilder benötigen Sie eine Reihe von Bibliotheken und Hilfsfunktionen. Die nachfolgende Codezelle importiert all diese Abhängigkeiten in das Projekt.

<div class="alert block" style="background-color: #D3D3D3;">
<b>Info:</b> Kürzlich wurde eine neue Version des Modells veröffentlicht: YOLOv10!</div>

<div class="alert alert-block alert-success">
<b>Auftrag:</b> Führen Sie die nachfolgende Codezelle mit einem Klick auf den Play-Button aus.</div>

![Image](../beispielbilder/grafiken/play.PNG)

In [ ]:
from ultralytics import YOLO                                 # Für Zugriff auf die YOLO-Modelle
import os                                                    # Zugriff auf die Funktionen des Betriebssystems

# Die folgenden Funktionen dienen der Unterstützung beim Erstellen der Daten und der Verwendung des Modells
###########################################################################################################
from helpers.image_helpers import capture_images             # Helferfunktion zum Aufnehmen der Bilder
from helpers.model_helpers import inference_webcam           # Helferfunktion zum Anwenden des Modells mit der Webcam
from helpers.model_helpers import choose_model               # Helferfunktion zur Auswahl des Modells
from helpers.model_helpers import set_training_config        # Helferfunktion zum Einstellen der Trainingskonfiguration
from helpers.file_handlers import prepare_folder_structure   # Helferfunktion zum Erstellen der benötigten Ordnerstruktur
from helpers.file_handlers import create_config              # Helferfunktion zum Erstellen der Konfigurationsdatei
from helpers.config import set_metadata                      # Helferfunktion zum Erstellen des Ordners
from helpers.config import set_image_data                    # Helferfunktion zum Erstellen des Ordners
TASK = 'CLS'



<div class="alert alert-block alert-warning">
<b>Optional:</b> Auch wenn Sie in diesem Notebook keinen eigenen Code schreiben müssen, ist für ein tieferes Verständnis der Vorgänge ein gewisses Verständnis von Code unabdingbar. Wenn es Sie interessiert oder Sie einfach gerne tiefer in die Materie eintauchen wollen, finden Sie im Ordner <code>notebooks/helpers</code> die Hilfsfunktionen, die wir Ihnen zur Verfügung stellen.</div>

## 1.2. Festlegen des Gruppennames

Ihre Datensätze und Modelle werden gespeichert und können Ihnen im Nachgang auf Wunsch zur Verfügung gestellt werden. 
Um die Identifikation zu gewährleisten, werden Sie nach Ausführen der nachfolgenden Codezelle dazu aufgefordert, einen Gruppennamen festzulegen. 

<div class="alert alert-block alert-success">
<b>Auftrag:</b> Führen Sie die nachfolgende Codezelle mit einem Klick auf den Play-Button aus.</div>

In [ ]:
GROUP, PATH = set_metadata(TASK)

## 1.3. Generieren der Bilddaten (15 min)

Nun beginnen wir mit dem Sammeln von Bildern. Die **Datenqualität** ist der entscheidende Faktor bei Machine-Learning-Projekten.

<div class="alert alert-block alert-success">
<b>Auftrag:</b> Machen Sie sich gemeinsam Gedanken zu einem Problem, welches Sie lösen wollen. Überlegen Sie außerdem <b>vorher</b> wie gute Daten dafür aussehen müssen. Denken Sie z.B. an Entfernung, Licht, Orientierung, Position, etc. Nehmen Sie sich dafür 15 Minuten Zeit!</div>

Sie sollten jetzt einen Plan haben zu:

* Anzahl der Bildklassen (z.B. defekt & nicht-defekt => 2 Klassen **oder** Aluprofil, Unterlegscheibe, Sechskant => 3 Klassen)
* Anzahl der Bilder pro Klasse (je mehr Klassen, desto mehr Bilder sollten Sie pro Klasse einplanen)
* Wie müssen die Bilder aussehen? (Beleuchtung, Ausrichtung der Objekte auf den Bildern, Entfernung zur Kamera, ...)

Die nachfolgende Codezelle hilft Ihnen dabei, Bilder aufzunehmen. <br>
Nach Ausführen der Zelle werden Sie dazu aufgefordert, den Klassennamen einzugeben, z.B. "aluprofil". <br>

Außerdem müssen Sie festlegen, wie viele Bilder für diese Klasse aufgenommen werden sollen, z.B. 100 <br>
Dann können Sie die Zeit zwischen zwei Aufnahmen festlegen. 

Jetzt beginnt die Webcam Bilder für diese Klasse aufzunehmen. Diese Bilder werden im Ordner `data/classification/IHR_GRUPPENNAME` gespeichert. 

<div class="alert alert-block alert-danger">
<b>Achtung:</b> Sie müssen die Codezelle für jede Klasse einmal ausführen.</div>


<div class="alert alert-block alert-success">
<b>Auftrag:</b> Beginnen Sie jetzt mit dem Sammeln der Daten.</div>

In [ ]:
NAME, NUM_IMGS, DELAY, DEVICE = set_image_data(TASK)

capture_images(num_imgs=NUM_IMGS,
               name=NAME,
               img_path=PATH,
               device=DEVICE,
               delay=DELAY,
               show=True)

## 1.4. Aufsetzen der Ordnerstruktur

Damit das Modell mit Ihren Daten arbeiten kann, müssen diese in einem bestimmten Format vorliegen. Aktuell befinden sich alle Bilder in einem Ordner. 

Hier ist die gewünschte Ordnerstruktur:

![Image](../beispielbilder/grafiken/ordnerstruktur.PNG)

Die Daten werden in **Trainings-**, **Validierungs-** und **Testdaten** aufgetrennt. 

Das Modell wird zunächst nur auf den **Trainingsdaten** trainiert und anschließend auf den **Validierungsdaten** überprüft. So wird eine **Überanpassung** auf die Trainingsdaten verhindert bzw. erkannt. Eine Überanpassung entspricht in etwa dem Auswendiglernen der Daten. Dann wäre das Modell zwar in der Lage, bereits bekannte Bilder fehlerfrei zu klassifizieren, könnte aber nicht auf ungesehene Daten **generalisieren**. Da das Modell aber in der Produktionsumgebung nicht mit exakt den Daten aus dem Training konfrontiert werden würde, ist eine Überanpassung unbedingt zu verhindern!

Erreicht das Modell nach dem Training die gewünschte Leistung, wird es am Ende noch ein letztes Mal auf den **Testdaten** überprüft, bevor es in die Produktionsumgebung überführt werden kann.

In jedem der `train`, `val` und `test` Ordner befinden sich jeweils weitere Unterordner für die einzelnen Klassen. Das `YOLO`-Modell wird anhand dieser Unterordner erkennen, welches **Label** ein Bild hat. 

Die ```prepare_folder_structure(PATH)```-Funktion stellt diese Ordnerstruktur automatisch für Sie her.

<div class="alert alert-block alert-warning">
<b>Optional:</b> Wenn Sie wissen möchten, wie diese Funktion dies tut, schauen Sie im Ordner <code>notebooks/helpers/misc.py</code> vorbei!</div>

Anschließend wird über die `create_config(PATH)`-Funktion die Konfigurationsdatei erstellt, die dem Modell sagt, wo die Daten zu finden sind.

<div class="alert alert-block alert-warning">
<b>Optional:</b> Auch diese finden Sie im Ordner <code>notebooks/helpers/misc.py</code>.</div>

<div class="alert alert-block alert-success">
<b>Auftrag:</b> Führen Sie die nachfolgende Codezelle mit einem Klick auf den Play-Button aus.</div>

In [ ]:
prepare_folder_structure(PATH, task=TASK)
create_config(PATH, task=TASK)

# 2. Modelltraining

Nachdem die Datenvorbereitung abgeschlossen ist, geht es nun daran, das Modell zu trainieren. 

Dieser Teil besteht aus zwei Schritten:

* Auswahl eines geeigneten Modells
* Trainieren dieses Modells

## 2.1. Modellauswahl

Die YOLO-Modelle zur Bildklassifierung kommen in verschiedenen Größen:

![Image](../beispielbilder/grafiken/yolo-cls.PNG)

Grundsätzlich gilt: Je größer das Modell, desto mächtiger ist es, aber desto langsamer ist es auch (sowohl während des Trainings als auch der Anwendung)

Hier müssen Sie entscheiden, ob es Ihnen wichtiger ist, ein sehr performantes Modell zu haben, welches aber etwas langsamer ist, oder ob es Ihnen vor allem auf Geschwindigkeit ankommt und Sie ein paar Abstriche in der Performance machen können.

Diese Entscheidung ist immer abhängig vom Einsatzgebiet des Modells. 

<div class="alert alert-block alert-success">
<b>Auftrag:</b> Führen Sie die nachfolgende Codezelle mit einem Klick auf den Play-Button aus. Sie werden anschließend dazu aufgerodert, ein Modell zu wählen.</div>

In [ ]:
model = choose_model(TASK)

## 2.2. Trainingsfunktion

Um das Modell zu trainieren müssen Sie diesem nur mitteilen, wo die Trainingsdaten zu finden sind. Dafür haben Sie vorher die Konfigurationsdatei erzeugt. 

```Python
results = model.train(data=PATH,        # Zeigt auf den Ordner mit der Konfigurationsdatei
                      epochs=EPOCHS,    # 1 Epoche = ein Durchlauf aller Daten durch das Modell
                      name=NAME,        # Name der einzelnen Trainingsdurchläufe
                      project=SAVE_DIR) # Name des Ordners in dem die Modellergebnisse gespeichert werden     
```

Beim Ausführen der nachfolgenden Codezelle werden Sie dazu aufgefordert die Anzahl der **Trainingsepochen** einzugeben. Wie lange das Modell trainieren soll ist immer Problemabhängig, aber für ein leichtes Problem sollten 20 - 30 Epochen vollkommen ausreichend sein. 

In [ ]:
EPOCHS, _, NAME, SAVE_DIR = set_training_config(PATH)

results = model.train(data=PATH, 
                      epochs=EPOCHS, 
                      name=NAME,
                      project=SAVE_DIR)

## 2.3. Evaluieren

Nachdem das Modell trainiert wurde, wurde im Ordner `GRUPPENNAME/runs/` automatisch ein Ordner für den Trainingsdurchlauf angelegt. Die Inhalte dieses Ordners sehen folgendermaßen aus:

![Image](../beispielbilder/grafiken/eval-cls.PNG)

Im `weights`-Ordner befinden sich zwei Dateien: `best.pt` und `last.pt`:

* `best.pt` ist der beste Zustand des Modells
* `last.pt` ist der letzte Zustand des Modells

Je nachdem ob Sie weitertrainieren möchten oder das Modell in die Produktionsumgebung überführen wollen, können Sie mit einer der beiden Dateien weiterarbeiten. 

<div class="alert alert-block alert-info">
<b>Frage:</b> Warum macht es im Hinblick auf Überanpassung Sinn, immer den besten Zustand des Modells zu speichern?</div>

Außerdem enthält der Ordner noch eine Reihe an anderen Dateien:

* Konfusionmatrizen
* results.png
* results.csv
* Trainings- und Validationsbatches

<div class="alert alert-block alert-info">
<b>Frage:</b> Nehmen Sie sich Zeit diese Dateien in Ruhe zu betrachten. Welche Informationen können Sie herauslesen?</div>

# 3. Verwenden des Modells

Nun kann das Modell verwendet werden.

Wir stellen Ihnen dafür die Hilfsfunktion `inference_webcam` zur Verfügung, welche die Webcam verwendet um einen Stream aufzunehmen und die Frames live zu klassifizieren.

Alternativ können Sie aber auch einzelne Bilder klassifizieren. Dafür müssten Sie die Bilddatei einfach dem Modell übergeben:

```Python
result = model('pfad/zum/bild.jpg')
```

<div class="alert alert-block alert-success">
<b>Auftrag:</b> Führen Sie die nachfolgende Codezelle mit einem Klick auf den Play-Button aus.</div>

<div class="alert alert-block alert-danger">
<b>Achtung:</b> Manchmal öffnet sich das Webcam-Fenster nicht automatisch oder die Webcam wird nicht erkannt. Im ersten Fall können Sie das Fenster unten in der Windows-Taskleiste auswählen. Im zweiten Fall führen Sie die Funktion einfach ein weiteres Mal aus.</div>

In [ ]:
inference_webcam(model=model,
                 device=0,
                 verbose=False)

# 4. Exportieren des Modells

Wenn in Ihrer Produktionsumgebung das `.pt` - Format verwendet werden sollte, dann sind Sie nun fertig. Allerdings können die trainierten Modell auch in eine ganze Reihe von verschiedenen Formaten exportiert werden, sodass diese nahtlos in Ihrer Produktionsumgebung verwendet werden können:

![Image](../beispielbilder/grafiken/formats.PNG)